# CW 10 Part 1
In this coursework, we aim to image a phase object after propagating for some distance in free space by using off-axis holography. 

You are given the code for loading an image, padding with zeroes and propagating for an arbitrary distance.

Then, the next steps are to
- interfere this field with an off-axis reference plane wave
- record the intensity
- extract incoming complex field by Fourier transforming and bringing the interference term to the center
- and propagate back

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from scipy.fft import fft2, ifft2, fftshift

img = data.camera()

img = np.pad(img, ((100, 100), (100, 100)), mode='constant')
img = img / 255.0  # Normalize to [0, 1]
ny, nx = img.shape
dx = dy = 10e-6  # Pixel size (10 microns)

phase = img * 2 * np.pi  # scale to [0, 2pi]
field = np.exp(1j * phase)  # complex field with phase encoding

def fresnel_propagate(u0, z, wavelength, dx):
    k = 2 * np.pi / wavelength
    ny, nx = u0.shape
    fx = np.fft.fftfreq(nx, d=dx)
    fy = np.fft.fftfreq(ny, d=dx)
    FX, FY = np.meshgrid(fx, fy)
    H = np.exp(-1j * np.pi * wavelength * z * (FX**2 + FY**2))
    U0 = fft2(u0)
    Uz = ifft2(U0 * H)
    return Uz

wavelength = 633e-9  # 633 nm
z = 0.03  # propagation distance in meters 
propagated_field = fresnel_propagate(field, z, wavelength, dx)

# Generate off-axis plane wave


x = (np.arange(nx) - nx // 2) * dx
y = (np.arange(ny) - ny // 2) * dy
X, Y = np.meshgrid(x, y)


theta_x = 3 * np.pi / 180  # 3 degrees off-axis in x
theta_y = 3 * np.pi / 180  # 3 degrees off-axis in y
